In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [2]:
class Config:
    TRAIN_PATH = '../inputs/train.csv'
    TEST_PATH = '../inputs/test.csv'
    TRAIN_LABELS = '../inputs/train_labels.csv'
    SAMPLE_SUBMISSION = '../inputs/sample_submission.csv'

In [3]:
def q2l(x):
    if x <= 3:
        return '0-4'
    if x <= 13:
        return '5-12'
    return '13-22'

In [36]:
event_features =  ['checkpoint_click','cutscene_click', 'person_click', 'navigate_click',
                   'observation_click', 'notification_click', 'object_click',
                   'object_hover', 'map_hover', 'map_click', 'notebook_click']
room_features = ['historicalsociety',
          'kohlcenter',
          'capitol_0',
          'humanecology',
          'drycleaner',
          'library',
          'capitol_1',
          'wildlife',
          'flaghouse','capitol_2']
num_features = [ 
        'page', 
        'room_coor_x', 
        'room_coor_y', 
        'screen_coor_x', 
        'screen_coor_y', 
        'hover_duration']
def groupby_apply(g):
    res = {}
    for col in num_features:
        res[f'{col}_mean'] = np.mean(g[col])
        res[f'{col}_max'] = np.max(g[col])
        res[f'{col}_min'] = np.min(g[col])
        res[f'{col}_std'] = np.std(g[col])
    elasped_time = g['elapsed_time'].values/1000
    level = g['level'].values
    res['duration'] = elasped_time.max() - elasped_time.min()
    for i in range(0,23):
        t = elasped_time[level==i]
        if len(t) > 0 :
            res[f'duration_level_{i}'] = t.max()-t.min()
        else:
            res[f'duration_level_{i}'] = 0
    res['text_fqid_null'] = pd.isnull(g['text_fqid']).sum()
    event_name_dict = Counter(g['event_name'].values)
    event_sequence = g['event_name'].values
    room_event_dict = Counter(g['room_event'].values)
    room_sequence = g['main_room'].values
    for col in event_features:
        res[f'{col}_sum'] = event_name_dict.get(col, 0)
    
    room_dict = Counter(g['main_room'].values)
    for col in room_features:
        res[f'{col}_sum'] = room_dict.get(col, 0)
    elapsed_time_diff_all = np.diff(elasped_time)
    res['elapsed_time_diff_mean'] = np.mean(elapsed_time_diff_all)
    res['elapsed_time_diff_std'] = np.std(elapsed_time_diff_all)
    res['elapsed_time_diff_max'] = np.max(elapsed_time_diff_all)
    res['elapsed_time_diff_min'] = np.min(elapsed_time_diff_all)
    res['elapsed_time_diff_positive'] = len(elapsed_time_diff_all[elapsed_time_diff_all>0])
    
    for col in event_features:
        elapsed_time_diff_event = elapsed_time_diff_all[event_sequence[1:]==col]
        elapsed_time_diff_event = elapsed_time_diff_event if len(elapsed_time_diff_event) > 0 else [0]
        res[f'elapsed_time_diff_{col}_mean'] = np.mean(elapsed_time_diff_event)
        res[f'elapsed_time_diff_{col}_max'] = np.max(elapsed_time_diff_event)
        res[f'elapsed_time_diff_{col}_min'] = np.min(elapsed_time_diff_event)
    return pd.Series(res)

In [37]:
def feature_engineering(df, meta):
    df['main_room'] = df['room_fqid'].str.split('.').str[1]
    df['room_event'] =  df['event_name']+'_' + df['main_room']
    X = df.groupby(['session','level_group']).apply(groupby_apply).reset_index()
    X = meta.merge(X,how='left', on=['session','level_group'])
    for i in range(1,19):
        X[f'q{i}'] = X['question'] == i
    return X

In [38]:
train_df = pd.read_csv(Config.TRAIN_PATH, usecols=lambda x: x not in ['fullscreen','hq','music'])
train_df['main_room'] = train_df['room_fqid'].str.split('.').str[1]
train_df['room_event'] =  train_df['event_name']+'_' + train_df['main_room']
train_labels = pd.read_csv(Config.TRAIN_LABELS)
train_df.rename(columns={'session_id':'session'},inplace=True)
train_labels['question'] = train_labels['session_id'].str.split('q').str[-1].astype('int')
train_labels['session'] = train_labels['session_id'].str.split('_').str[0].astype('int64')
train_labels['level_group'] = train_labels['question'].apply(lambda x: q2l(x))

In [39]:
df1 = train_df.loc[train_df.level_group=='0-4']
X1 = df1.groupby(['session']).apply(groupby_apply).reset_index()
X1['level_group'] = '0-4'
del df1
gc.collect()

37

In [40]:
df2 = train_df.loc[(train_df.level_group=='0-4')|(train_df.level_group=='5-12')]
X2 = df2.groupby(['session']).apply(groupby_apply).reset_index()
X2['level_group'] = '5-12'
del df2
gc.collect()

0

In [41]:
X3 = train_df.groupby(['session']).apply(groupby_apply).reset_index()
X3['level_group'] = '13-22'
gc.collect()

0

In [42]:
X = pd.concat([X1,X2,X3])
X = train_labels.merge(X,how='left', on=['session','level_group'])
for i in range(1,19):
    X[f'q{i}'] = X['question'] == i

In [43]:
FEATURES = X.columns[5:]
len(FEATURES)

126

In [71]:
n_splits=5
gkf = GroupKFold(n_splits=n_splits)
oof = np.zeros(X.shape[0])
models = {}

# COMPUTE CV SCORE WITH 5 GROUP K FOLD
for i, (train_index, valid_index) in enumerate(gkf.split(X, groups=X['session'])):
    print('#'*25)
    print('### Fold',i+1)
    print('#'*25)

    # xgb_params = {
    #     'objective' : 'binary:logistic',
    #     'eval_metric':'logloss',
    #     'learning_rate': 0.01,
    #     'max_depth': 5,
    #     'n_estimators': 3000,
    #     'early_stopping_rounds': 50,
    #     'subsample':0.8,
    #     'colsample_bytree': 0.8,
    #     'tree_method': 'gpu_hist',
    #     'seed':42,
    #     'use_label_encoder' : False}

    X_train = X.iloc[train_index][FEATURES]
    X_valid = X.iloc[valid_index][FEATURES]
    y_train = X.iloc[train_index]['correct'].values
    y_valid = X.iloc[valid_index]['correct'].values
        # TRAIN MODEL
    # clf =  XGBClassifier(**xgb_params)
    clf = CatBoostClassifier(
    iterations=3000, 
    learning_rate=0.03,
    max_depth=5,
    subsample=0.8,
    task_type='GPU',
    early_stopping_rounds=50,
        random_state=42,
        auto_class_weights='Balanced',
        bootstrap_type='Poisson'
    #loss_function='CrossEntropy'
)
    clf.fit(X_train.astype('float32'), y_train,
            eval_set=[(X_valid.astype('float32'),y_valid)],
            verbose=100)
    # print(f'({clf.best_ntree_limit}), ',end='')

        # SAVE MODEL, PREDICT VALID OOF
    models[i] = clf
    oof[valid_index] = clf.predict_proba(X_valid)[:,1]

    print()

#########################
### Fold 1
#########################
0:	learn: 0.6881789	test: 0.6880348	best: 0.6880348 (0)	total: 12.2ms	remaining: 36.7s
100:	learn: 0.5825245	test: 0.5778433	best: 0.5778433 (100)	total: 606ms	remaining: 17.4s
200:	learn: 0.5685858	test: 0.5637905	best: 0.5637905 (200)	total: 1.22s	remaining: 17s
300:	learn: 0.5628842	test: 0.5588872	best: 0.5588872 (300)	total: 1.83s	remaining: 16.4s
400:	learn: 0.5593680	test: 0.5564495	best: 0.5564495 (400)	total: 2.43s	remaining: 15.8s
500:	learn: 0.5563858	test: 0.5546111	best: 0.5546111 (500)	total: 3.04s	remaining: 15.2s
600:	learn: 0.5538504	test: 0.5533750	best: 0.5533750 (600)	total: 3.67s	remaining: 14.6s
700:	learn: 0.5517604	test: 0.5525798	best: 0.5525798 (700)	total: 4.27s	remaining: 14s
800:	learn: 0.5498532	test: 0.5518729	best: 0.5518655 (799)	total: 4.87s	remaining: 13.4s
900:	learn: 0.5480341	test: 0.5514117	best: 0.5514094 (898)	total: 5.48s	remaining: 12.8s
1000:	learn: 0.5463559	test: 0.5510189	best:

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

def plot_feature_importance(importance,names,model_type):

    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df = fi_df.head(50)
    #Define size of bar plot
    plt.figure(figsize=(20,10))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
feat_imp = {}
for k, v in models.items():
    for x, y in zip(v.feature_importances_, v.feature_names_):
        if y not in feat_imp:
            feat_imp[y] = x
        else:
            feat_imp[y]+=x
plot_feature_importance(list(feat_imp.values()),list(feat_imp.keys()),'')

In [ ]:
scores = []; thresholds = []
best_score = 0; best_threshold = 0

for threshold in np.arange(0.4,0.9,0.01):
    print(f'{threshold:.02f}, ',end='')
    preds = (oof>threshold).astype('int')
    m = f1_score(X['correct'], preds, average='macro')
    scores.append(m)
    thresholds.append(threshold)
    if m>best_score:
        best_score = m
        best_threshold = threshold

In [ ]:
import matplotlib.pyplot as plt

# PLOT THRESHOLD VS. F1_SCORE
plt.figure(figsize=(20,5))
plt.plot(thresholds,scores,'-o',color='blue')
plt.scatter([best_threshold], [best_score], color='blue', s=300, alpha=1)
plt.xlabel('Threshold',size=14)
plt.ylabel('Validation F1 Score',size=14)
plt.title(f'Threshold vs. F1_Score with Best F1_Score = {best_score:.5f} at Best Threshold = {best_threshold:.3}',size=18)
plt.show()

In [53]:
from catboost import CatBoostClassifier